In [1]:
import tensorflow as tf
import numpy as np
from utils import *
from VDSH_S import *
from __future__ import print_function

filename = 'dataset/ng20.tfidf.mat'
data = Load_Dataset(filename)

latent_dim = 32
sess = get_session("1", 0.10)
model = VDSH_S(sess, latent_dim, data.n_feas, data.n_tags, keep_prob=0.9)

In [2]:
# create an optimizer
learning_rate=0.00
decay_rate = 0.96
#decay_step = 10000
step = tf.Variable(0, trainable=False)  
lr = tf.train.exponential_decay(learning_rate, 
                                step, 
                                10000, 
                                decay_rate, 
                                staircase=True, name="lr")

my_optimizer = tf.train.AdamOptimizer(learning_rate=lr) \
                     .minimize(model.cost, global_step=step)
    
init = tf.global_variables_initializer()
model.sess.run(init)

In [5]:
total_epoch = 25
kl_weight = 0.
kl_inc = 1 / 5000. # set the annealing rate for KL loss

for epoch in xrange(total_epoch):
    epoch_loss = []
    for i in range(len(data.train)):
        # get doc
        doc = data.train[i]
        labels = data.gnd_train[i]
        word_indice = np.where(doc > 0)[0]
        
        # indices
        opt, loss = model.sess.run((my_optimizer, model.cost), 
                                    feed_dict={model.input_bow: doc.reshape((-1, data.n_feas)),
                                               model.input_bow_idx: word_indice,
                                               model.labels: labels.reshape((-1, data.n_tags)),
                                               model.kl_weight: kl_weight})
        
        kl_weight = min(kl_weight + kl_inc, 1.0)
        epoch_loss.append(loss)
        
        if i % 50 == 0:
            print(loss)
        #    print("\rEpoch:{}/{} {}/{}: Loss:{:.3f} AvgLoss:{:.3f}".format(epoch+1, total_epoch, i, len(data.train), loss, np.mean(epoch_loss)), end='')

[ 234.43606567]
[ 200.41218567]
[ 259.75820923]
[ 261.67160034]
[ 283.50524902]
[ 282.36785889]
[ 319.03665161]
[ 348.29473877]
[ 333.30761719]
[ 366.98446655]
[ 361.33285522]
[ 353.81585693]
[ 411.0357666]
[ 415.95758057]
[ 429.42080688]
[ 424.347229]


KeyboardInterrupt: 

In [ ]:
# run experiment here
zTrain = model.transform(data.train)
zTest = model.transform(data.test)
zTrain = np.array(zTrain)
zTest = np.array(zTest)
medHash = MedianHashing()
cbTrain = medHash.fit_transform(zTrain)
cbTest = medHash.transform(zTest)

TopK=100
print('Retrieve Top{} candidates using hamming distance'.format(TopK))
run_topK_retrieval_experiment(cbTrain, cbTest, data.gnd_train, data.gnd_test, TopK)